In [1]:
from ccxt import okex5,bitget,bybit,binanceusdm,Exchange
import pandas as pd
from random import random,randint,uniform
from datetime import datetime,timedelta

In [2]:
def generate_open_order(exchange:Exchange):
    exchange.load_markets()
    data_list = [symbol for symbol in exchange.symbols if ':USDT' in symbol]
    random_open_order_symbols = [data_list[i] for i in sorted([int(random()*len(data_list)) for _ in range(25)])]
    entry_data = {}
    order_id_count = 1
    for symbol in random_open_order_symbols:
        side = randint(0,1)
        symbol_history_df = pd.DataFrame(exchange.fetch_ohlcv(symbol, '15m', limit=1000))
        symbol_min_value = exchange.markets[symbol]['limits']['amount']['min']
        symbol_max_value = exchange.markets[symbol]['limits']['amount']['max']
        if symbol_max_value == None or symbol_min_value == None:
            symbol_max_value, symbol_min_value = 100,500
        quantity = uniform(symbol_min_value,symbol_max_value)
        if quantity*symbol_history_df[4][1] > 1000:
            quantity = round(1000/symbol_history_df[4][1],4)
        if side == 0:
            most_lowest_price_index = symbol_history_df[4].idxmin()
            entry_data[symbol] = {
                'order_id': order_id_count,
                'side': 'buy',
                'entry_price': symbol_history_df[4][most_lowest_price_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_lowest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
            }
        else:
            most_highest_price_index = symbol_history_df[4].idxmax()
            entry_data[symbol] = {
                'order_id': order_id_count,
                'side': 'sell',
                'entry_price': symbol_history_df[4][most_highest_price_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][most_highest_price_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
            }
        order_id_count += 1
    return entry_data

def Caculate_Unrealized_Pnl(open_orders_data:dict,exchange:Exchange):
    for symbol in open_orders_data:
        symbol_history_df = pd.DataFrame(exchange.fetch_ohlcv(symbol, '1m', limit=10))
        most_recent_price_index = symbol_history_df[4].idxmax()
        most_recent_price = symbol_history_df[4][most_recent_price_index]
        if open_orders_data[symbol]['side'] == 'buy':
            open_orders_data[symbol]['Unrealized'] = round((most_recent_price - open_orders_data[symbol]['entry_price'])*open_orders_data[symbol]['entry_quantity'],2)
        else:
            open_orders_data[symbol]['Unrealized'] = round((open_orders_data[symbol]['entry_price'] - most_recent_price)*open_orders_data[symbol]['entry_quantity'],2)
    return open_orders_data

def generate_close_order(exchange:Exchange,total_trade_num:int,win_trade_num:int,time_start:datetime,pick_up_number:int=10):
    time_end = datetime.now() - timedelta(days=1)
    time_end = time_end.isoformat()
    time_end = exchange.parse8601(time_end)
    time_start = exchange.parse8601(time_start.isoformat())
    exchange.load_markets()
    data_list = [symbol for symbol in exchange.symbols if ':USDT' in symbol]
    random_close_order_symbols = [data_list[i] for i in sorted([int(random()*len(data_list)) for _ in range(pick_up_number)])]
    order_id_count = 1
    close_order_data = {}
    while len(close_order_data) < total_trade_num:
        symbol = random_close_order_symbols[int(random()*len(random_close_order_symbols))]
        side = randint(0,1)
        symbol_history_df = []
        time_temp = time_start
        while time_temp < time_end:
            symbol_history_df.extend(exchange.fetch_ohlcv(symbol, '1h', time_temp, limit=1000))
            time_temp = symbol_history_df[-1][0] + 3600000
        symbol_history_df = pd.DataFrame(symbol_history_df)
        symbol_min_value = exchange.markets[symbol]['limits']['amount']['min']
        symbol_max_value = exchange.markets[symbol]['limits']['amount']['max']
        if symbol_max_value == None or symbol_min_value == None:
            symbol_max_value, symbol_min_value = 100,500
        quantity = uniform(symbol_min_value,symbol_max_value)
        if quantity*symbol_history_df[4][1] > 1000:
            quantity = round(1000/symbol_history_df[4][1],4)
        if win_trade_num != 0:
            random_start_index = int(random()*(len(symbol_history_df)-100))
            side = 'buy' if randint(0,1) == 0 else 'sell'
            if side == 'buy':
                random_end_index = symbol_history_df[random_start_index:random_start_index+1000][4].idxmax()
            else:
                random_end_index = symbol_history_df[random_start_index:random_start_index+1000][4].idxmin()
            close_order_data[order_id_count] = {
                'symbol': symbol,
                'side': side,
                'entry_price': symbol_history_df[4][random_start_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][random_start_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
                'exit_price': symbol_history_df[4][random_end_index],
                'exit_time': datetime.fromtimestamp(symbol_history_df[0][random_end_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'exit_quantity': quantity,
                'Realized_Pnl': round((symbol_history_df[4][random_end_index] - symbol_history_df[4][random_start_index])*quantity,2) if side == 'buy' else round((symbol_history_df[4][random_start_index] - symbol_history_df[4][random_end_index])*quantity,2),
                'Unrealized_Pnl': 0,
            }
            order_id_count += 1
            win_trade_num -= 1
        else:
            random_start_index = int(random()*(len(symbol_history_df)-100))
            side = 'buy' if randint(0,1) == 0 else 'sell'
            if side == 'buy':
                random_end_index = symbol_history_df[random_start_index:random_start_index+200][4].idxmin()
            else:
                random_end_index = symbol_history_df[random_start_index:random_start_index+200][4].idxmax()
            close_order_data[order_id_count] = {
                'symbol': symbol,
                'side': side,
                'entry_price': symbol_history_df[4][random_start_index],
                'entry_time': datetime.fromtimestamp(symbol_history_df[0][random_start_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'entry_quantity': quantity,
                'entry_leverage': randint(1,25),
                'entry_type': 'market',
                'entry_class_SF': 'swap',
                'exchange': exchange.id,
                'exit_price': symbol_history_df[4][random_end_index],
                'exit_time': datetime.fromtimestamp(symbol_history_df[0][random_end_index]/1000).strftime("%Y-%m-%d %H:%M:%S"),
                'exit_quantity': quantity,
                'Realized_Pnl': round((symbol_history_df[4][random_end_index] - symbol_history_df[4][random_start_index])*quantity,2) if side == 'buy' else round((symbol_history_df[4][random_start_index] - symbol_history_df[4][random_end_index])*quantity,2),
                'Unrealized_Pnl': 0,
                
            }
            order_id_count += 1
    return close_order_data

In [3]:
binance_usdm = binanceusdm({
    'option': {
        "defaultType": "swap",
    },
})
entry_data = generate_open_order(binance_usdm)
order_data = Caculate_Unrealized_Pnl(entry_data,binance_usdm)
order_data_df = pd.DataFrame(order_data).T
order_data_df

,order_id,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
1000SHIB/USDT:USDT,1,buy,0.010026,2023-03-12 23:30:00,98048.8283,18,market,swap,binanceusdm,84.13
1000XEC/USDT:USDT,2,sell,0.03329,2023-03-18 08:45:00,37257.8241,16,market,swap,binanceusdm,74.52
ALGO/USDT:USDT,3,buy,0.1816,2023-03-12 23:30:00,5376.3441,13,market,swap,binanceusdm,260.75
ALICE/USDT:USDT,4,sell,1.749,2023-03-18 15:45:00,740.7407,4,market,swap,binanceusdm,129.63
ALPHA/USDT:USDT,5,sell,0.12074,2023-03-18 21:30:00,11014.4289,5,market,swap,binanceusdm,97.7
APT/USDT:USDT,7,sell,14.394,2023-03-14 22:30:00,97.012,5,market,swap,binanceusdm,225.26
BAKE/USDT:USDT,8,buy,0.1591,2023-03-12 05:45:00,6285.3551,5,market,swap,binanceusdm,202.39
BLUEBIRD/USDT:USDT,9,sell,9.032,2023-03-18 14:45:00,143.4103,20,market,swap,binanceusdm,51.2
CFX/USDT:USDT,10,buy,0.1412,2023-03-13 00:30:00,7002.8011,13,market,swap,binanceusdm,1916.67
CVX/USDT:USDT,12,sell,5.998,2023-03-15 09:30:00,201.6942,10,market,swap,binanceusdm,85.11


In [4]:
close_data = generate_close_order(binance_usdm,10,9,datetime(2023,1,1))
close_data_df = pd.DataFrame(close_data).T
close_data_df

,symbol,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,exit_price,exit_time,exit_quantity,Realized_Pnl,Unrealized_Pnl
1,ZEC/USDT:USDT,sell,43.88,2023-02-28 03:00:00,26.9397,7,market,swap,binanceusdm,30.02,2023-03-11 15:00:00,26.9397,373.38,0
2,NKN/USDT:USDT,sell,0.09839,2023-03-09 20:00:00,12655.024,16,market,swap,binanceusdm,0.08408,2023-03-10 18:00:00,12655.024,181.09,0
3,CRV/USDT:USDT,buy,0.801,2023-01-15 17:00:00,1897.5332,5,market,swap,binanceusdm,1.29,2023-02-21 08:00:00,1897.5332,927.89,0
4,CRV/USDT:USDT,buy,0.544,2023-01-06 09:00:00,1897.5332,14,market,swap,binanceusdm,1.209,2023-02-08 21:00:00,1897.5332,1261.86,0
5,ANT/USDT:USDT,sell,3.117,2023-02-19 14:00:00,448.2295,14,market,swap,binanceusdm,1.945,2023-03-10 18:00:00,448.2295,525.32,0
6,ANT/USDT:USDT,sell,3.209,2023-02-02 16:00:00,448.2295,6,market,swap,binanceusdm,1.945,2023-03-10 18:00:00,448.2295,566.56,0
7,XRP/USDT:USDT,sell,0.3757,2023-03-04 12:00:00,2951.5939,1,market,swap,binanceusdm,0.3506,2023-03-13 00:00:00,2951.5939,74.09,0
8,STORJ/USDT:USDT,sell,0.475,2023-02-23 16:00:00,4108.4634,22,market,swap,binanceusdm,0.2896,2023-03-10 18:00:00,4108.4634,761.71,0
9,BTCDOM/USDT:USDT,sell,1581.8,2023-03-18 09:00:00,0.7328,11,market,swap,binanceusdm,1578.3,2023-03-18 11:00:00,0.7328,2.56,0
10,STORJ/USDT:USDT,buy,0.3372,2023-03-16 08:00:00,4108.4634,3,market,swap,binanceusdm,0.3372,2023-03-16 08:00:00,4108.4634,0.0,0


In [5]:
close_data

{1: {'symbol': 'ZEC/USDT:USDT',
  'side': 'sell',
  'entry_price': 43.88,
  'entry_time': '2023-02-28 03:00:00',
  'entry_quantity': 26.9397,
  'entry_leverage': 7,
  'entry_type': 'market',
  'entry_class_SF': 'swap',
  'exchange': 'binanceusdm',
  'exit_price': 30.02,
  'exit_time': '2023-03-11 15:00:00',
  'exit_quantity': 26.9397,
  'Realized_Pnl': 373.38,
  'Unrealized_Pnl': 0},
 2: {'symbol': 'NKN/USDT:USDT',
  'side': 'sell',
  'entry_price': 0.09839,
  'entry_time': '2023-03-09 20:00:00',
  'entry_quantity': 12655.024,
  'entry_leverage': 16,
  'entry_type': 'market',
  'entry_class_SF': 'swap',
  'exchange': 'binanceusdm',
  'exit_price': 0.08408,
  'exit_time': '2023-03-10 18:00:00',
  'exit_quantity': 12655.024,
  'Realized_Pnl': 181.09,
  'Unrealized_Pnl': 0},
 3: {'symbol': 'CRV/USDT:USDT',
  'side': 'buy',
  'entry_price': 0.801,
  'entry_time': '2023-01-15 17:00:00',
  'entry_quantity': 1897.5332,
  'entry_leverage': 5,
  'entry_type': 'market',
  'entry_class_SF': 'swap

In [6]:
bybit_ex = bybit({
    "option":{
        "defaultType":"swap",
    }
})
bybit_ex.load_markets()
entry_data = generate_open_order(bybit_ex)
order_data = Caculate_Unrealized_Pnl(entry_data,bybit_ex)
order_data_df = pd.DataFrame(order_data).T
order_data_df

,order_id,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
ADA/USDT:USDT,1,buy,0.3296,2023-03-21 15:00:00,2940.3117,10,market,swap,bybit,148.19
ALPHA/USDT:USDT,2,sell,0.1183,2023-03-20 16:45:00,8737.4399,22,market,swap,bybit,55.48
BAND/USDT:USDT,3,sell,1.824,2023-03-20 18:45:00,524.090645,5,market,swap,bybit,33.54
BEL/USDT:USDT,4,buy,0.6124,2023-03-21 16:30:00,1505.7973,11,market,swap,bybit,49.99
BICO/USDT:USDT,5,buy,0.3909,2023-03-21 16:15:00,147.797619,24,market,swap,bybit,3.19
COTI/USDT:USDT,6,buy,0.06951,2023-03-21 16:30:00,13406.6229,22,market,swap,bybit,67.7
DOGE/USDT:USDT,7,buy,0.07048,2023-03-21 15:00:00,13557.4837,14,market,swap,bybit,69.14
ENJ/USDT:USDT,8,buy,0.384,2023-03-21 16:15:00,2389.4863,6,market,swap,bybit,46.59
ETH/USDT:USDT,9,sell,1818.94,2023-03-21 21:45:00,0.5687,9,market,swap,bybit,13.52
FET/USDT:USDT,10,sell,0.41775,2023-03-20 15:45:00,2461.5385,12,market,swap,bybit,46.03


In [7]:
close_data = generate_close_order(bybit_ex,10,9,datetime(2023,1,1))
close_data_df = pd.DataFrame(close_data).T
close_data_df

,symbol,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,exit_price,exit_time,exit_quantity,Realized_Pnl,Unrealized_Pnl
1,IOTA/USDT:USDT,buy,0.194,2023-03-13 01:00:00,5934.7181,21,market,swap,bybit,0.2315,2023-03-18 11:00:00,5934.7181,222.55,0
2,1INCH/USDT:USDT,buy,0.521,2023-03-05 00:00:00,2604.1667,2,market,swap,bybit,0.5665,2023-03-19 16:00:00,2604.1667,118.49,0
3,1000BTT/USDT:USDT,sell,0.000744,2023-01-30 15:00:00,1607717.0418,20,market,swap,bybit,0.000591,2023-03-11 19:00:00,1607717.0418,245.98,0
4,OMG/USDT:USDT,sell,1.222,2023-01-19 11:00:00,974.6589,4,market,swap,bybit,1.216,2023-01-19 20:00:00,974.6589,5.85,0
5,1000BTT/USDT:USDT,sell,0.000694,2023-02-11 00:00:00,1113745.623601,2,market,swap,bybit,0.000591,2023-03-11 19:00:00,1113745.623601,114.72,0
6,JST/USDT:USDT,buy,0.02103,2023-01-09 17:00:00,50403.2258,25,market,swap,bybit,0.03215,2023-02-07 11:00:00,50403.2258,560.48,0
7,BLUR/USDT:USDT,sell,0.5798,2023-03-16 22:00:00,1303.781,2,market,swap,bybit,0.5403,2023-03-21 00:00:00,1303.781,51.5,0
8,BICO/USDT:USDT,buy,0.2681,2023-01-08 02:00:00,1997.036754,23,market,swap,bybit,0.4383,2023-01-30 20:00:00,1997.036754,339.9,0
9,JST/USDT:USDT,buy,0.02944,2023-02-26 16:00:00,39188.499032,15,market,swap,bybit,0.03064,2023-02-27 08:00:00,39188.499032,47.03,0
10,DYDX/USDT:USDT,buy,2.465,2023-02-10 21:00:00,913.242,18,market,swap,bybit,2.355,2023-02-13 17:00:00,913.242,-100.46,0


In [8]:
okex5_ex = okex5({
    "option":{
        "defaultType":"swap",
    }
})
okex5_ex.load_markets()
entry_data = generate_open_order(okex5_ex)
order_data = Caculate_Unrealized_Pnl(entry_data,okex5_ex)
order_data_df = pd.DataFrame(order_data).T
order_data_df

,order_id,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,Unrealized
AGLD/USDT:USDT,1,buy,0.4099,2023-03-21 15:00:00,271.816241,3,market,swap,okex5,6.55
BAND/USDT:USDT,2,buy,1.66,2023-03-21 15:00:00,149.210073,8,market,swap,okex5,15.07
COMP/USDT:USDT,3,buy,42.74,2023-03-21 17:15:00,21.9106,13,market,swap,okex5,46.01
DASH/USDT:USDT,4,sell,58.99,2023-03-22 05:15:00,17.7336,2,market,swap,okex5,20.39
ETC/USDT:USDT-230324,5,buy,19.195,2023-03-21 15:00:00,47.1054,8,market,swap,okex5,67.36
JST/USDT:USDT,6,sell,0.02807,2023-03-20 07:15:00,315.147204,3,market,swap,okex5,0.33
KISHU/USDT:USDT,7,sell,0.0,2023-03-20 03:15:00,113.937851,14,market,swap,okex5,0.0
LINK/USDT:USDT,8,buy,6.873,2023-03-21 16:00:00,141.7234,13,market,swap,okex5,97.36
LTC/USDT:USDT-230331,9,buy,76.93,2023-03-21 15:00:00,11.9646,22,market,swap,okex5,52.29
LUNA/USDT:USDT,10,buy,1.3639,2023-03-21 17:15:00,184.647626,15,market,swap,okex5,12.78


In [9]:
close_data = generate_close_order(okex5_ex,10,9,datetime(2023,1,1))
close_data_df = pd.DataFrame(close_data).T
close_data_df

,symbol,side,entry_price,entry_time,entry_quantity,entry_leverage,entry_type,entry_class_SF,exchange,exit_price,exit_time,exit_quantity,Realized_Pnl,Unrealized_Pnl
1,BNT/USDT:USDT,buy,0.4281,2023-02-11 07:00:00,301.403278,7,market,swap,okex5,0.6083,2023-03-21 21:00:00,301.403278,54.31,0
2,ONT/USDT:USDT,buy,0.2291,2023-02-07 23:00:00,463.333207,21,market,swap,okex5,0.263,2023-03-18 08:00:00,463.333207,15.71,0
3,CORE/USDT:USDT,buy,2.10265,2023-03-15 20:00:00,246.1944,18,market,swap,okex5,2.44,2023-03-21 18:00:00,246.1944,83.05,0
4,NEAR/USDT:USDT,buy,1.78,2023-03-10 23:00:00,166.415578,1,market,swap,okex5,2.2,2023-03-18 20:00:00,166.415578,69.89,0
5,BNT/USDT:USDT,buy,0.4673,2023-03-16 22:00:00,335.738013,21,market,swap,okex5,0.6083,2023-03-21 21:00:00,335.738013,47.34,0
6,SLP/USDT:USDT,sell,0.00338,2023-02-08 05:00:00,338.523816,13,market,swap,okex5,0.00238,2023-03-10 18:00:00,338.523816,0.34,0
7,GMT/USDT:USDT,buy,0.432,2023-02-11 07:00:00,296.646463,10,market,swap,okex5,0.4933,2023-03-19 11:00:00,296.646463,18.18,0
8,NEAR/USDT:USDT,sell,2.61,2023-02-09 09:00:00,115.384164,11,market,swap,okex5,1.69,2023-03-10 18:00:00,115.384164,106.15,0
9,NEAR/USDT:USDT,sell,1.813,2023-03-11 14:00:00,380.361727,13,market,swap,okex5,1.704,2023-03-11 23:00:00,380.361727,41.46,0
10,NEAR/USDT:USDT,sell,2.564,2023-02-08 14:00:00,425.7131,13,market,swap,okex5,2.757,2023-02-08 22:00:00,425.7131,-82.16,0
